# Monte Carlo On policy
## Sources 
- Reinforcement Learning : An Introduction, by Richard Sutton and Andrew Barto

Toujours basé sur le Blackjack, nous allons rechercher une politique optimum.


In [1]:
import gymnasium as gym
import pandas as pd
from collections import defaultdict
import random
import numpy as np

Création de l'environnement Blackjack

In [2]:
env = gym.make('Blackjack-v1')

Dictionnaire pour stocker les Q values (action-valeur)

In [3]:
Q = defaultdict(float)

Dictionnaire pour le total des retours G poour chaque couple s,a

In [4]:
total_return = defaultdict(float)

Dictionnaire pour le cumul du nombre de passage pour tout les s,a

In [5]:
N = defaultdict(int)

## Définissons une politique epsilon-greedy
Paramètres d'entrés   
    - Q fonction (s,a)   
    - L'état courant   
    - epsilon   
 
Paramètre de sortie    
    - a action choisie

In [18]:
def epsilon_greedy_policy(state,Q,epsilon, pPrint=False):

    # tirer un nombre aléatoire entre 0 et 1
    # si ce nombre est inférieur à epsilon, tirer une action au hasard
    # Sinon en partant de Q, pour l'état state, choisir l'action qui est la plus rémunératrice
    # 10-15 lignes
   
    
    random_nb = random.uniform(0,1)
    if random_nb < epsilon:
        return env.action_space.sample()
    else:
        max_q = -1
        max_a = 0
        for a in range(env.action_space.n):
            if (state, a) in Q:
                if Q[(state, a)] > max_q:
                    max_q = Q[(state, a)] 
                    max_a = a
        return max_a
        
    #return action


### chatgpt

In [69]:
def epsilon_greedy_policy_2(state, Q, epsilon, pPrint=False):
    # Draw a random number between 0 and 1
    random_number = np.random.rand()

    if pPrint:
        print(f'random_number: {random_number}')
        print(f'epsilon: {epsilon}')
        print(f'Q: {Q}')
        print(f'state: {state}')
        
    # Filter the actions for the given state
    actions_for_state = {action: Q[(state, action)] for action in range(len(Q)) if (state, action) in Q}
    
    if pPrint:
        print(f'actions_for_state: {actions_for_state}')
    
    # If the random number is less than epsilon, choose a random action (exploration)
    if random_number < epsilon:
        action = np.random.choice(list(actions_for_state.keys()))  # Random action from the action space
        
        if pPrint:
            print(f'[IF] Action: {action}')
        
    else:
        # Otherwise, choose the action with the highest Q value (exploitation)
        action = max(actions_for_state, key=actions_for_state.get)  # Select the action with the maximum Q value
        if pPrint:
            print(f'[ELSE] Action: {action}')
    
    return action  # Return the chosen action


In [8]:
Z=defaultdict(float)
Z[(1,0)] = 10
Z[(1,1)] = 12
action = epsilon_greedy_policy(1,Z,0, pPrint=True) 
print (f'action: {action}')
#assert epsilon_greedy_policy(1,Z,0) ==1

action: 1


In [56]:
Z 

defaultdict(float, {(1, 0): 10, (1, 1): 12, 1: 0.0})

In [67]:

max_key = max(Z, key=Z.get)
max_value = Z[max_key]
print(f"Max key: {max_key}, Max value: {max_value}")

Max key: (1, 1), Max value: 12


In [51]:
for k in Z:
    print(f'k: {k}')

k: (1, 0)
k: (1, 1)
k: 1


## Générateur d'épisode
Entrée :
- Q fonction
- espilon pour notre politique aléatoire

Sortie :
- notre épisode sous la forma s,a ,s'

In [12]:
num_timesteps = 100
#epsilon = 0.01

In [13]:
def generate_episode(Q,epsilon):
    
    episode = []
    
    state = env.reset()[0]
    
    for t in range(num_timesteps):
        
        # Sélection d'une action en fonction de notre politique
        action = epsilon_greedy_policy(state,Q,epsilon)
        
        # envoie de l'action à l'environnement pour retour (s_, r, done)
        next_state, reward, done, truncated, info = env.step(action)
        
        # stockage dans la liste du triplet (état, action, récompense)
        episode.append((state, action, reward))
        
        if done:
            break
            
        state = next_state

    return episode

***
# Calcul de la politique optimale



In [15]:
#num_iterations = 50000
num_iterations = 3
espilon = 0.5

Nous allons initialiser une politique random au démarrage.      
Selon le modèle GPI, Notre Q fonction va tendre vers l'optimale et donc notre politique également.<br>
Pour simplifier et parce que l'intéret est limité dans le cas du blackjack, nous ferons abstraction de gamma.

In [17]:
random.seed(1)
for i in range(num_iterations ):
    
    # on génére un épisode
    episode = generate_episode(Q,espilon)
    
    # on stocker les pairs s,a de l'épisode
    all_state_action_pairs = [(s, a) for (s,a,r) in episode]
    
    # on stocke les récompense
    rewards = [r for (s,a,r) in episode]

    # Pour chaque t de l'épisode 
    for t, (state, action, reward) in enumerate(episode):

        # First visit : on ne prend en compte que le premier passage s,a
        if not (state, action) in all_state_action_pairs[0:t]:
            
            # Calcul de G avec y = 1
            G = sum(rewards[t:]) 
            
            # Accumulate G for the (state, action) pair
            total_return[(state, action)] = total_return.get((state, action), 0) + G
            
            
            # Comptage du nombre de passage
            N[(state, action)] = N.get((state, action), 0) + 1

            # Calcul de Q value (s,a) par la moyenne des G cumulés sur N
            # Votre code : mettre à jour Q, ce qui mettrait à jour notre politique
            # 1 ligne
             # Update the Q-value (state, action) as the average of cumulative returns            
            Q[(state, action)] = total_return[(state, action)] / N[(state, action)]


# Stockage du résultat dans un Dataframe

In [21]:
type(Q.items())

dict_items

In [23]:
print(len(Q.items()))

7


In [24]:
df = pd.DataFrame(Q.items(),columns=['state_action pair','value'])

In [25]:
print(df.shape)

(7, 2)


In [26]:
df.head(11)

,state_action pair,value
0,"((8, 5, 0), 0)",-1.0
1,"((18, 2, 0), 0)",1.0
2,"((11, 3, 0), 0)",-1.0
3,"((16, 10, 0), 0)",-1.0
4,"((19, 7, 0), 0)",1.0
5,"((19, 7, 0), 1)",0.0
6,"((17, 4, 0), 0)",-1.0


Nous avons un Q optimal ou proche.   
Notre politique est basée sur une approche gloutonne de Q donc elle même optimale !

***
## Exemple
J'ai en main 19 points (sans as) et le croupier 5 points visibles.

In [58]:
df[ df['state_action pair'] == ((19,5,False),0) ]

,state_action pair,value
226,"((19, 5, 0), 0)",0.467101


In [65]:
df[ df['state_action pair'] == ((10,10,False),1) ]

,state_action pair,value
207,"((10, 10, 0), 1)",-0.220868


## Ma politique va donc choisir un stand (0) car la valeur de fonction d'action-état est la plus haute !

In [62]:
epsilon_greedy_policy((10, 10, 0), Q, 0)

1